# Programación Funcional en Python

La programación funcional (PF) es la descripción de nuestra solución o algoritmo en términos de relaciones entre funciones.
Si bien Python no es un lenguaje de PF, adopta algunas características de los mismos debido a que la PF permite:
* Pensar la solución como transformaciones sucesivas de los datos, lo que simplifica el diseño
* Mejora el diseño, ya que hay una descomposición clara del problema, y cada parte se encuentra encapsulada.
* Mejora la legibilidad (a la larga) ya que los algoritmos "revelan la intención"
* Facilita el testing ya que los trozos de código a testear son más simples

Es por eso que es sano tratar de aplicar (con el debido criterio) características de programación funcional en nuestro código.

In [11]:
#como parámetro
def decirle_a(persona, accion):
    print(accion(persona)) 
    
def saludar(persona):
    return 'hola ' + persona

def despedir(persona):
    return 'chau ' + persona

decirle_a('Carlos', despedir)
decirle_a('Ana', saludar)

print(despedir) 

chau Carlos
hola Ana
<function despedir at 0x7f8dc078e510>


## Listas y diccionarios por comprensión

Vamos a comenzar por algo simple.
Les parece familiar el siguiente patrón?
```python
lst = []
for x in iterable:
    lst.append(hacer_algo(x))    
```

Y éste?
```python
lst = []
for x in iterable:
    if es_de_alguna_manera(x):
        lst.append(x)    
```
Seguramente los vieron ya en alguna variante. Para esos casos, que son muy comunes, Python provee las listas y diccionarios por comprensión, que son una manera *pythonica* de escribir lo mismo, pero que se entienda mejor.
Por ejemplo, los fragmentos de código que vieron arriba se pueden escribir como
```python
lst = [hacer_algo(x) for x in iterable]
lst = [x for x in iterable if es_de_alguna_manera(x)]
```
También se puede aplicar a diccionarios!

```python
d = {k:v for k, v in enum(iterable)}
```

## Funciones de Orden Superior

Un concepto clave para entender PF es que las *funciones también son datos*, es decir, se pueden pasar como parámetro, devolver como resultado, asignar a un identificador, etc. Si una función recibe a otra como parámetro, o devuelve una función como resultado, decimos que esa función es de *orden superior*. Si bien el nombre asusta un poco, es posible que usemos varias funciones de ese estilo cuando trabajemos con Python

### Decoradores

Los decoradores son funciones que *envuelven* a otras funciones, agregándoles funcionalidad extra. De esta manera podemos aumentar las capacidades del código que tenemos sin necesidad de modificarlo.

La forma básica de un decorador es

```python
def decorador(funcion):
    def envolvedor(argumentos):
        #Hacer algo con funcion(argumentos)
    return envolvedor
```

Por ejemplo, imagínense el caso en el que estamos debugguendo código, y queremos ver en pantalla, cuándo entramos a una función y con qué parámetros.

In [25]:
def logging_decorator(output_stream):
    "Decorador configurable para hacer logging"
    def decorator(f):
        "Devuelve función decorada"
        def wrapper(*args, **kwargs):
            "Función contenedora"
            print("Ejecutando función {} con argumentos {} y {}".format(f.__name__, args, kwargs))
            resultado = f(*args, **kwargs)
            print("imprimiendo %s en %s" %(resultado, output_stream))
            
        return wrapper
    
    return decorator

@logging_decorator("pantalla")
def ident(x):
    return x

@logging_decorator("pantalla")
def devuelve_42():
    return 42

ident("hola")
devuelve_42()



Ejecutando función ident con argumentos ('hola',) y {}
imprimiendo hola en pantalla
Ejecutando función devuelve_42 con argumentos () y {}
imprimiendo 42 en pantalla


No es necesario programar nuestros decoradores. Usualmente usaremos los que vengan provistos en los módulos y paquetes que usemos. Sin embargo, está bueno saber cómo funcionan las cosas que usamos, no?

A continuación veremos más ejemplo del uso de decoradores.

### Módulos `itertools` y `functools` y `operator`

El módulo `itertools` provee facilidades para trabajar con secuencias de datos potencialmente infinitas.
El módulo `functools` provee herramientas para trabajar sobre funciones.
En `operator` encontramos una interface para trabajar con los operadores *built-in*, es decir, con los que nos da el lenguaje.

En el siguiente ejemplo vemos el uso de `functools.partial`, el cual nos sirve para evaluar parcialmente cualquier función, es decir, poder "presetear" algunos de los parámetros de la función.

In [13]:
import functools


def myfunc(a, b=2):
    "Docstring for myfunc()."
    print('  called myfunc with:', (a, b))


def show_details(name, f, is_partial=False):
    "Show details of a callable object."
    print('{}:'.format(name))
    print('  object:', f)
    if not is_partial:
        print('  __name__:', f.__name__)
    if is_partial:
        print('  func:', f.func)
        print('  args:', f.args)
        print('  keywords:', f.keywords)
    return


show_details('myfunc', myfunc)
myfunc('a', 3)
print()

# Set a different default value for 'b', but require
# the caller to provide 'a'.
p1 = functools.partial(myfunc, b=4)
show_details('partial with named default', p1, True)
p1('passing a')
p1('override b', b=5)
print()

# Set default values for both 'a' and 'b'.
p2 = functools.partial(myfunc, 'default a', b=99)
show_details('partial with defaults', p2, True)
p2()
p2(b='override b')
print()

print('Insufficient arguments:')
p1()


myfunc:
  object: <function myfunc at 0x7f8dc078e950>
  __name__: myfunc
  called myfunc with: ('a', 3)

partial with named default:
  object: functools.partial(<function myfunc at 0x7f8dc078e950>, b=4)
  func: <function myfunc at 0x7f8dc078e950>
  args: ()
  keywords: {'b': 4}
  called myfunc with: ('passing a', 4)
  called myfunc with: ('override b', 5)

partial with defaults:
  object: functools.partial(<function myfunc at 0x7f8dc078e950>, 'default a', b=99)
  func: <function myfunc at 0x7f8dc078e950>
  args: ('default a',)
  keywords: {'b': 99}
  called myfunc with: ('default a', 99)
  called myfunc with: ('default a', 'override b')

Insufficient arguments:


TypeError: myfunc() missing 1 required positional argument: 'a'

Otra función que nos provee `functools` es la de facilitarnos la posibilidad de definir orden entre nuestros propios tipos de datos mediante el decorador `fuctools.total_ordering`.

In [15]:
import functools
import inspect
from pprint import pprint


@functools.total_ordering
class MyObject:

    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        print('  testing __eq__({}, {})'.format(
            self.val, other.val))
        return self.val == other.val

    def __gt__(self, other):
        print('  testing __gt__({}, {})'.format(
            self.val, other.val))
        return self.val > other.val


print('Methods:\n')
pprint(inspect.getmembers(MyObject, inspect.isfunction))

a = MyObject(1)
b = MyObject(2)

print('\nComparisons:')
for expr in ['a < b', 'a <= b', 'a == b', 'a >= b', 'a > b']:
    print('\n{:<6}:'.format(expr))
    result = eval(expr)
    print('  result of {}: {}'.format(expr, result))

Methods:

[('__eq__', <function MyObject.__eq__ at 0x7f8dc07f5f28>),
 ('__ge__', <function _ge_from_gt at 0x7f8dccb45d90>),
 ('__gt__', <function MyObject.__gt__ at 0x7f8dc07f5ea0>),
 ('__init__', <function MyObject.__init__ at 0x7f8dc07f5d08>),
 ('__le__', <function _le_from_gt at 0x7f8dccb45e18>),
 ('__lt__', <function _lt_from_gt at 0x7f8dccb45d08>)]

Comparisons:

a < b :
  testing __gt__(1, 2)
  testing __eq__(1, 2)
  result of a < b: True

a <= b:
  testing __gt__(1, 2)
  result of a <= b: True

a == b:
  testing __eq__(1, 2)
  result of a == b: False

a >= b:
  testing __gt__(1, 2)
  testing __eq__(1, 2)
  result of a >= b: False

a > b :
  testing __gt__(1, 2)
  result of a > b: False


Para terminar de hablar de functools, mencionaremos el decorador `single_dispatch` que viene genial para cuando el comportamiento de nuestra función depende del tipo del parámetro.

In [16]:
import functools


@functools.singledispatch
def myfunc(arg):
    print('default myfunc({!r})'.format(arg))


@myfunc.register(int)
def myfunc_int(arg):
    print('myfunc_int({})'.format(arg))


@myfunc.register(list)
def myfunc_list(arg):
    print('myfunc_list()')
    for item in arg:
        print('  {}'.format(item))


myfunc('string argument')
myfunc(1)
myfunc(2.3)
myfunc(['a', 'b', 'c'])


default myfunc('string argument')
myfunc_int(1)
default myfunc(2.3)
myfunc_list()
  a
  b
  c


El módulo `itertools` nos brinda muchas herramientas para trabajar de manera eficiente con secuencias de datos. El módulo es extenso y tiene muchas aplicaciones. Veremos algunas de ellas, pero consulten la documentación del módulo para ver otras.

In [26]:
import functools
from itertools import *
import operator
import pprint


@functools.total_ordering
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __repr__(self):
        return '({}, {})'.format(self.x, self.y)

    def __eq__(self, other):
        return (self.x, self.y) == (other.x, other.y)

    def __gt__(self, other):
        return (self.x, self.y) > (other.x, other.y)


# Create a dataset of Point instances
data = list(map(Point,
                cycle(islice(count(), 3)),
                islice(count(), 7)))
print('Data:')
pprint.pprint(data, width=35)
print()

# Try to group the unsorted data based on X values
print('Grouped, unsorted:')
for k, g in groupby(data, operator.attrgetter('x')):
    print(k, list(g))
print()

# Sort the data
data.sort()
print('Sorted:')
pprint.pprint(data, width=35)
print()

# Group the sorted data based on X values
print('Grouped, sorted:')
for k, g in groupby(data, operator.attrgetter('x')):
    print(k, list(g))
print()

Data:
[(0, 0),
 (1, 1),
 (2, 2),
 (0, 3),
 (1, 4),
 (2, 5),
 (0, 6)]

Grouped, unsorted:
0 [(0, 0)]
1 [(1, 1)]
2 [(2, 2)]
0 [(0, 3)]
1 [(1, 4)]
2 [(2, 5)]
0 [(0, 6)]

Sorted:
[(0, 0),
 (0, 3),
 (0, 6),
 (1, 1),
 (1, 4),
 (2, 2),
 (2, 5)]

Grouped, sorted:
0 [(0, 0), (0, 3), (0, 6)]
1 [(1, 1), (1, 4)]
2 [(2, 2), (2, 5)]



El módulo `operator` nos permite utilizar a los operadores built-in de Python para adaptarlos a nuestros propósitos.

### Generadores y Evaluación Perezosa

Usualmente Python trata de evaluar el resultado de algo inmediatamente. Para algunos casos, esto no está bueno. Por ejemplo, trabajar con listas *potencialmente* infinitas. En estos casos, lo mejor es que no se evalúe nada hasta que no sea necesario. Es decir, saber tanto de algo como necesitemos, pero no más. A ésto se lo llama **Evaluación Perezosa** ("Lazy Evaluation"). Python no soporta este modo de evaluación, pero se puede emular con una herramienta muy copada que son los **generadores**.
Un generador es un objeto iterable que una vez instanciado *calcula* su siguiente elemento en cada llamada `next()`, en vez de tenerlo almacenado en memoria como otras estructuras de datos. La palabra clave que distingue a un generador es `yield` que es un return que espera una próxima llamada para continuar desde donde quedó la última vez.

In [7]:
#cuenta 
def contador():
    x = 1
    while True:
        yield x
        x += 1
c = contador()
print(next(c))
print(next(c))
print(next(c))

1
2
3


Otra cosa piola de Python es que existen las **generator expressions** que se parecen a las listas por comprensión, pero devuelven un generador.

In [9]:
c = (i for i in range(10))
c
print(next(c))
print(next(c))

0
1


## ¿Dónde puedo aprender más?


- Documentación oficial
    + Functional Programming Howto
    + módulos itertools y functools
- Python Module of the Week
    + módulos itertools, operator y functools
- "Functional Programming in Python", de David Mertz, ed O'Reilly
- Paquetes como OSlash, PyMonad, etc.
- Internet
